# Analysing satelite data

## THIS NOTEBOOK IS NOT USED IN CURRENT VERSION OF THE WORKSHOP

Just like any other image data

Most of the examples are based on http://www.scipy-lectures.org/advanced/image_processing/

## Data source


## Bands description

https://earth.esa.int/web/sentinel/user-guides/sentinel-2-msi/resolutions/spatial

![](https://earth.esa.int/image/image_gallery?uuid=c5fa6c3e-2978-4fb8-ac95-3be9c5171be2&groupId=247904&t=1345630320883)

SENTINEL-2 10 m spatial resolution bands: B2 (490 nm), B3 (560 nm), B4 (665 nm) and B8 (842 nm)

![](https://earth.esa.int/image/image_gallery?uuid=15dad96b-be6a-4b04-931d-d8c4db39e9e2&groupId=247904&t=1345630328076)

SENTINEL-2 20 m spatial resolution bands: B5 (705 nm), B6 (740 nm), B7 (783 nm), B8a (865 nm), B11 (1610 nm) and B12 (2190 nm)

![](https://earth.esa.int/image/image_gallery?uuid=f6117fbe-1513-4a84-acc4-845e14e5c876&groupId=247904&t=1345630315020)

SENTINEL-2 60 m spatial resolution bands: B1 (443 nm), B9 (940 nm) and B10 (1375 nm)

## Displaying color composition

In [ ]:
import rasterio
import rasterio.plot

blue_source = rasterio.open("../../data/sentinel/S2A_OPER_MSI_L1C_TL_EPA__20160605T113933_B02.tiff")
blue = blue_source.read()
rasterio.plot.show(blue, cmap="gray")

In [ ]:
green_source = rasterio.open("../../data/sentinel/S2A_OPER_MSI_L1C_TL_EPA__20160605T113933_B03.tiff")
green = green_source.read()
rasterio.plot.show(green, cmap="gray")

red_source = rasterio.open("../../data/sentinel/S2A_OPER_MSI_L1C_TL_EPA__20160605T113933_B04.tiff")
red = red_source.read()
rasterio.plot.show(blue, cmap="gray")

In [ ]:
import rasterio.plot
import numpy as np

imgdata = np.array([red, green, blue])
rasterio.plot.show(imgdata)

![](../../images/rgb-sentinel2.png)

## Calculate NDVI
```
NDVI=(NIR−VIS)/(NIR+VIS)
```

In [ ]:
nir_source = rasterio.open("../../data/sentinel/S2A_OPER_MSI_L1C_TL_EPA__20160605T113933_B08.tiff")
nir = nir_source.read()

![](../../images/nir-sentinel2.png)

In [ ]:
ndvi = (nir.astype(float)-red.astype(float))/(nir.astype(float)+red.astype(float))

rasterio.plot.show(ndvi)
rasterio.plot.show_hist(ndvi)

In [ ]:
# save file
kwargs = red_source.meta
kwargs.update(dtype=rasterio.float64, count=1, compress='lzw')
with rasterio.open('../../outputs/ndvi.tif', 'w', **kwargs) as dst:
    dst.write_band(1, ndvi[0].astype(rasterio.float64))

![](../../images/ndvi-sentinel2.png)

## Image filtering

Local filters: replace the value of pixels by a function of the values of neighboring pixels.
Neighbourhood: square (choose size), disk, or more complicated structuring element.

### Ready to use filters

* Gaussian
* Median
* Sobel
* ...

#### Gaussian filter


In [ ]:
from scipy import ndimage

blurred_red = ndimage.gaussian_filter(red, 3)
rasterio.plot.show(red[0][300:400, 500:600])
rasterio.plot.show(blurred_red[0][300:400, 500:600])

#### Median filter

In [ ]:
sharpen_red = ndimage.median_filter(red, 3)


rasterio.plot.show(sharpen_red[0][300:400, 500:600])

#### Sobel filter

In [ ]:
sharpen_ndvi = ndimage.median_filter(ndvi, 3)
sx = ndimage.sobel(ndvi[0], axis=0, mode='constant')
sy = ndimage.sobel(ndvi[0], axis=1, mode='constant')
sob = np.hypot(sx, sy)
rasterio.plot.show(ndvi[0][300:400, 500:600])
rasterio.plot.show(sob[300:400, 500:600])

### Segmentation 

In [ ]:
rasterio.plot.show_hist(ndvi, bins=50)

In [ ]:
water = ndvi[0] < 0.2
rasterio.plot.show(water)
print(water.sum())

In [ ]:
# Remove small white regions
open_img = ndimage.binary_opening(water)
# Remove small black hole
water_clear = ndimage.binary_closing(open_img)
rasterio.plot.show(water_clear)
print(water_clear.sum())